In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np

from queries.getTestsStarts import ret as problemsStarts
from queries.getSubmissionsTime import ret as problemsEnds
from queries.getSubTimeForOpenAssess import ret as problemsAssessEnds
from queries.getVideoStartAndEnd import ret as videoStartsAndEnds
from queries.getSeqEvent import ret as seqEvents
from queries.getTextsInteractions import ret as textsInteractions
from queries.getHintsTime import ret as hintsTime 

from getThemeStartTimeForUser import getThemeStarts
from getProblemRes import grades

In [3]:
graded,ungraded=grades()
graded['percent']=graded['grade']/graded['maxGrade']
ungraded['percent']=ungraded['grade']/ungraded['maxGrade']

GET http://localhost:9200/online_courses_full/_search?scroll=5m&size=1000 [status:N/A request:2.032s]
Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\site-packages\urllib3\connection.py", line 171, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "C:\Users\User\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 79, in create_connection
    raise err
  File "C:\Users\User\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 69, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 172, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)


GET http://localhost:9200/online_courses_full/_search?scroll=5m&size=1000 [status:N/A request:2.007s]
Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\site-packages\urllib3\connection.py", line 171, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "C:\Users\User\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 79, in create_connection
    raise err
  File "C:\Users\User\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 69, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\Anaconda3\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 172, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)


ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x000001AEBCCE8588>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x000001AEBCCE8588>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение)

In [ ]:
problemsStartsDF=pd.DataFrame(problemsStarts())
problemsStartsDF['action'] = problemsStartsDF['problem_type']+' '+problemsStartsDF['problem_id']
problemsStartsDF=problemsStartsDF.loc[:,['course_id','time','user_id','action']]

In [ ]:
problemsEndsDF=pd.DataFrame(problemsEnds())
problemsEndsDF['action'] = 'end ungraded '+problemsEndsDF['problem_id']
problemsEndsDF=problemsEndsDF.loc[:,['course_id','time','user_id','action']]

In [ ]:
problemsAssessEndsDF=pd.DataFrame(problemsAssessEnds())
problemsAssessEndsDF['action'] = 'end graded '+problemsAssessEndsDF['problem_id']
problemsAssessEndsDF=problemsAssessEndsDF.loc[:,['course_id','time','user_id','action']]

In [ ]:
videoStartsAndEndsDF=pd.DataFrame(videoStartsAndEnds())
videoStartsAndEndsDF=videoStartsAndEndsDF.loc[:,['course_id','time','user_id']]
videoStartsAndEndsDF['action'] = 'video'

In [ ]:
textsInteractionsDF=pd.DataFrame(textsInteractions())
textsInteractionsDF['course_id']=videoStartsAndEndsDF['course_id'][0]
textsInteractionsDF['action']='text'

In [ ]:
getThemeStartsDF=pd.DataFrame(getThemeStarts())
getThemeStartsDF=getThemeStartsDF.loc[:,['course_id','time','user_id']]
getThemeStartsDF['action']='theme start'

In [ ]:
hintsTimeDF=pd.DataFrame(hintsTime())
hintsTimeDF['action']='hint'

In [ ]:
seqTimeDF=pd.DataFrame(seqEvents())

In [ ]:
DF=pd.concat([problemsStartsDF, problemsEndsDF,problemsAssessEndsDF,videoStartsAndEndsDF,textsInteractionsDF,getThemeStartsDF,hintsTimeDF,seqTimeDF],ignore_index=True,sort=False)

In [ ]:
DF.to_csv(path_or_buf='users_sequence.csv',index=False)

In [ ]:
#DF = pd.read_csv('users_sequence.csv', sep=',', encoding='utf-8')

In [ ]:
users=np.unique(DF['user_id'])

In [ ]:
studyModelsDF=[]
problemModelsDF=[]

In [ ]:
for user in users:
    userActions=DF.loc[DF['user_id']==user].sort_values(by='time')
    sequence=userActions['action'].values
    
    sequenceForModel=[]
    themeArr=[]
    
    #gradedPosEnd=[]
    #gradedPosStart=[]
    gradedSequences=[]
    
    #ungradedPosEnd=[]
    #ungradedPosStart=[]
    ungradedSequences=[]
    
    studySequence=[]
    
    for step in sequence:
        if step == 'theme start':
            if len(themeArr)>1:
                sequenceForModel.append(themeArr)
            themeArr=['theme start']
        elif len(themeArr)>=1:
            if step!=themeArr[-1] or step.find('end')!=-1:
                themeArr.append(step)
    sequenceForModel.append(themeArr)
    
    for actionsArr in sequenceForModel:
        if 'theme start' in actionsArr:
            actionsArr.remove('theme start')
            
        gradedPosEnd=[elem for elem in actionsArr if(elem.find('end graded')!=-1)]   
        gradedPosEnd=np.unique(gradedPosEnd)        
       
        if len(gradedPosEnd)>0:            
            for end in gradedPosEnd:
                endPosInSeq=[i for i, elem in enumerate(actionsArr) if elem == end]
                startPosInSeq=[i for i, elem in enumerate(actionsArr) if (elem.find(end.split('end ')[-1])!=-1)]
                startPosInSeq.sort()

                for endIndex in endPosInSeq:
                    if endIndex!=0:
                        startIndex=startPosInSeq[startPosInSeq.index(endIndex)-1]
                    else:
                        startIndex=endIndex
                gradedSequences.append(actionsArr[startIndex:(endIndex+1)]) 
                del actionsArr[startIndex:endIndex]     
                
        ungradedPosEnd=[elem  for elem in actionsArr if(elem.find('end ungraded')!=-1)]  
        ungradedPosEnd=np.unique(ungradedPosEnd)
            
        if len(ungradedPosEnd)>0:
            for end in ungradedPosEnd:
                endPosInSeq=[i for i, elem in enumerate(actionsArr) if elem == end]
                startPosInSeq=[i for i, elem in enumerate(actionsArr) if (elem == end.split('end ')[-1] or elem == end)]
                startPosInSeq.sort()

                for endIndex in endPosInSeq:
                    if endIndex!=0:
                        startIndex=startPosInSeq[startPosInSeq.index(endIndex)-1]
                    else:
                        startIndex=endIndex
                ungradedSequences.append(actionsArr[startIndex:(endIndex+1)])    
                del actionsArr[startIndex:endIndex]  
            
        for elem in actionsArr:
            if elem.find('graded')!=-1 and elem.find('end')==-1:
                while elem in actionsArr:
                    actionsArr.remove(elem)
                
        while 'seq' in actionsArr:
            actionsArr.remove('seq')              
        while 'seq' in gradedSequences:
            gradedSequences.remove('seq')              
        while [] in gradedSequences:
            gradedSequences.remove([])  
        while 'seq' in ungradedSequences:
            ungradedSequences.remove('seq')  
        while [] in ungradedSequences:
            ungradedSequences.remove([])  
            
    while [] in sequenceForModel:
        sequenceForModel.remove([])

    studySeq=[]
    probSeq=[]
    for array in sequenceForModel:
        array.insert(0,'TS')
        array.append('TE')
        studySeq.extend(array)
    for array in gradedSequences:
        array.insert(0,'PS')
        array.append('PE')
        probSeq.extend(array)
    for array in ungradedSequences:
        array.insert(0,'PS')
        array.append('PE')
        probSeq.extend(array)
        
       
    studySeq=['problem' if x.find('end')!=-1 else x for x in studySeq]  
        
    studyStatesCounter=pd.DataFrame(columns=['TS','video','text','problem','TE'])
    studyStatesCounter['TS']=[studySeq.count('TS')]
    studyStatesCounter['TE']=[studySeq.count('TE')]
    studyStatesCounter['video']=[studySeq.count('video')]
    studyStatesCounter['text']=[studySeq.count('text')]
    studyStatesCounter['problem']=[studySeq.count('problem')]
    
    study=pd.DataFrame(columns=['states','TS','video','text','problem','TE'])
    study['states']=['TS','video','text','problem','TE']
    study=study.set_index('states')
    study=study.fillna(0)
    study['TE']['TE']=1
    
    oldState=False
    
    while 'hint' in studySeq:
        studySeq.remove('hint')
        
    for state in studySeq:
        if state.find('grade')!=-1:
            continue
        if state!='TS' and oldState!='TE' and oldState!=False:
            study[state][oldState]+=1
        oldState=state    
    
    study=study.transpose()
    for col in study.columns:
        if studyStatesCounter[col][0]>0:
            study[col]/=studyStatesCounter[col][0]
    study=study.transpose()
    
    study['TS']='*'    
    study=study.transpose()
    study['TE']='*'
    study=study.transpose()
    study['TE']['TS']='*'    
    
    study=study.values
    
    studyModel=[]
    for array in study:
        studyModel.extend(array)
    while '*' in studyModel:
        studyModel.remove('*')    
        
    studyModelsDF.append(studyModel)
    
    probSeq=['CH' if x.find('end')!=-1 else x for x in probSeq] 
    probSeq=['PV' if x.find('graded')!=-1 else x for x in probSeq] 
    
    problemStatesCounter=pd.DataFrame(columns=['PS','video','text','hint','PV','CH','PE'])
    problemStatesCounter['PS']=[probSeq.count('PS')]
    problemStatesCounter['PE']=[probSeq.count('PS')]
    problemStatesCounter['video']=[probSeq.count('video')]
    problemStatesCounter['text']=[probSeq.count('text')]
    problemStatesCounter['hint']=[probSeq.count('hint')]
    problemStatesCounter['CH']=[probSeq.count('CH')]
    problemStatesCounter['PV']=[probSeq.count('PV')] 
    
    problem=pd.DataFrame(columns=['states','PS','video','text','hint','PV','CH','PE'])
    problem['states']=['PS','video','text','hint','PV','CH','PE']
    problem=problem.set_index('states')
    problem=problem.fillna(0)
    problem['PE']['PE']=1
    
    oldState=False
    while 'seq' in probSeq:
        probSeq.remove('seq')
        
    for state in probSeq:
        if state!='PS' and oldState!='PE' and oldState!=False:
            problem[state][oldState]+=1
        oldState=state
        
    problem=problem.transpose()
    for col in problem.columns:
        if problemStatesCounter[col][0]>0:
            problem[col]/=problemStatesCounter[col][0]
    problem=problem.transpose()
    
    problem['PS']='*'
    tmp=problem['PE']['CH']
    problem['PE']='*'
    problem['PE']['CH']=tmp
    
    problem=problem.transpose()
    problem['PE']='*'
    problem=problem.transpose()
    
    problem=problem.values
    
    problemModel=[]
    for array in problem:
        problemModel.extend(array)
    while '*' in problemModel:
        problemModel.remove('*')
    problemModelShort=[]
    weight=problemStatesCounter['video'][0]+problemStatesCounter['text'][0]+problemStatesCounter['hint'][0]
    problemModelShort.append(weight)
    if weight>0:
        problemModelShort.append(problemStatesCounter['video'][0])
        problemModelShort.append(problemStatesCounter['text'][0])
        problemModelShort.append(problemStatesCounter['hint'][0])
    else:
        problemModelShort.append(0)
        problemModelShort.append(0)
        problemModelShort.append(0)
        
    
    percentGR=list(graded.loc[graded['user_id']==user]['percent'])
    percentUN=list(ungraded.loc[ungraded['user_id']==user]['percent'])
    
    if len(percentGR)==1:
        percentGR=percentGR[0]
    else:
        percentGR=-1
        
    if len(percentUN)==1:
        percentUN=percentUN[0]
    else:
        percentUN=-1
        
    problemModelShort.append(percentGR)
    problemModelShort.append(percentUN)
    problemModelShort.append(problemStatesCounter['CH'][0])

    problemModelsDF.append(problemModelShort)
    
studyModelsDF=pd.DataFrame(studyModelsDF,columns=['TS_V','TS_T','TS_P',
                                                  'V_V','V_T','V_P','V_TE',
                                                  'T_V','T_T','T_P','T_TE',
                                                  'P_V','P_T','P_P','P_TE'])
#studyModelsDF.to_csv(path_or_buf='users_study_seq_model.csv',index=False)
"""
problemModelsDF=pd.DataFrame(problemModelsDF,columns=['PS_V','PS_T','PS_H','PS_PV','PS_CH',
                                                      'V_V','V_T','V_H','V_PV','V_CH',
                                                      'T_V','T_T','T_H','T_PV','T_CH',
                                                      'H_V','H_T','H_H','H_PV','H_CH',
                                                      'PV_V','PV_T','PV_H','PV_PV','PV_CH',
                                                      'CH_V','CH_T','CH_H','CH_PV','CH_CH','CH_PE',
                                                      'vid_part','text_part','hint_part','graded_percent','ungraded_percent'     
                                                     ]) 
"""
#problemModelsDF.to_csv(path_or_buf='users_prob_seq_model.csv',index=False)
problemModelsDF=pd.DataFrame(problemModelsDF,columns=[
                                                      'actionsNumber','vidNum','textNum','hintNum','graded_percent','ungraded_percent','checksNum'     
                                                     ]) 

In [ ]:
studyModelsDF=studyModelsDF.drop_duplicates()
studyModelsDF.to_csv(path_or_buf='users_study_seq_model.csv',index=False)

In [ ]:
#problemModelsDF=problemModelsDF.drop_duplicates()
problemModelsDF.to_csv(path_or_buf='users_prob_seq_model.csv',index=False)

In [ ]:
problemModelsDF.shape